<a href="https://colab.research.google.com/github/zacSimo/PysparkAdvanced/blob/main/pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=678a8bc770eef7266e946cf01b317083afe8c5b8f1356de2481b8bbc3f67d93e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
import pyspark.sql.functions as f

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Practice').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 14:49:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
sd_df = spark.read.csv('/kaggle/input/fifa-18-demo-player-dataset/PlayerAttributeData.csv', header=True, inferSchema=True)


In [6]:
sd_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Aggression: string (nullable = true)
 |-- Agility: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- Ball control: string (nullable = true)
 |-- Composure: string (nullable = true)
 |-- Crossing: string (nullable = true)
 |-- Curve: string (nullable = true)
 |-- Dribbling: string (nullable = true)
 |-- Finishing: string (nullable = true)
 |-- Free kick accuracy: string (nullable = true)
 |-- GK diving: string (nullable = true)
 |-- GK handling: string (nullable = true)
 |-- GK kicking: string (nullable = true)
 |-- GK positioning: string (nullable = true)
 |-- GK reflexes: string (nullable = true)
 |-- Heading accuracy: string (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Interceptions: string (nullable = true)
 |-- Jumping: string (nullable = true)
 |-- Long passing: string (nullable = true)
 |-- Long shots: string (nullable = true)
 |-- Marking: string (nullable = 

In [7]:
sd_df.count()

17981

#####Apache spark memory

In [8]:
sd_broad_df = f.broadcast(sd_df)

In [9]:
# UDF

def my_func(code: str) -> str:

  return bdData.value.get(code)




In [10]:
from pyspark.sql.types import StringType,ArrayType ,StructType,StructField,IntegerType
spark.udf.register("my_function", my_func, StringType())

<function __main__.my_func(code: str) -> str>

In [11]:
# see how many cores available for spark engine

spark.sparkContext.defaultParallelism

4

In [12]:
# generate ramdom data

import random

randomlist = random.sample(range(1, 40),10)

In [13]:
print(randomlist)

[15, 4, 14, 5, 25, 11, 35, 7, 19, 37]


In [14]:
# parallelize and take it into 4 partitions

rdd1 = spark.sparkContext.parallelize(randomlist, 4)

In [15]:
rdd1

ParallelCollectionRDD[17] at readRDDFromFile at PythonRDD.scala:289

In [16]:
# show exactly the same data as created

rdd1.collect()

[15, 4, 14, 5, 25, 11, 35, 7, 19, 37]

In [17]:
# to see the numbers of partitions

rdd1.getNumPartitions()

4

In [18]:
# to see the data in partitions

rdd1.glom().collect()

[[15, 4], [14, 5], [25, 11], [35, 7, 19, 37]]

In [19]:
rdd1.take(rdd1.count()-1)

[15, 4, 14, 5, 25, 11, 35, 7, 19]

In [20]:
rdd1.glom().collect()[3]

[35, 7, 19, 37]

In [21]:
rdd1.top(3)

[37, 35, 25]

In [22]:
rdd_map = rdd1.map(lambda x: x*2)

rdd_map.collect()

[30, 8, 28, 10, 50, 22, 70, 14, 38, 74]

In [23]:
def myfunc(x):

  if x%2 == 0:

    return x * 2

  else:

    return x*3



rdd_map_2 = rdd1.map(myfunc)

rdd_map_2.collect()

[45, 8, 28, 15, 75, 33, 105, 21, 57, 111]

In [24]:
rdd_filter = rdd1.filter(lambda x: x%2 == 0)

rdd_filter.collect()

[4, 14]

In [25]:
# flatmap collect values into a single list

# used when we want aggregation of the result (reduce)

rdd_flatmap = rdd1.flatMap(lambda x: (x, x*10))

rdd_flatmap.collect()

[15,
 150,
 4,
 40,
 14,
 140,
 5,
 50,
 25,
 250,
 11,
 110,
 35,
 350,
 7,
 70,
 19,
 190,
 37,
 370]

In [26]:
rdd_map_3 = rdd1.map(lambda x: [x, x*10])

rdd_map_3.collect()

[[15, 150],
 [4, 40],
 [14, 140],
 [5, 50],
 [25, 250],
 [11, 110],
 [35, 350],
 [7, 70],
 [19, 190],
 [37, 370]]

In [27]:
rdd_map_3.glom().collect()

[[[15, 150], [4, 40]],
 [[14, 140], [5, 50]],
 [[25, 250], [11, 110]],
 [[35, 350], [7, 70], [19, 190], [37, 370]]]

In [28]:
print(rdd1.max())

print(rdd1.min())

print(rdd1.mean())

print(rdd1.sum())

print(rdd1.stdev())

37
4
17.2
172
11.196428001822724


In [29]:
# mapPartitions() map a function into each partitions

def my_partition_func(iterator):

  sum = 0

  for i in iterator:

    sum += i



  yield sum



rdd_map_partition = rdd1.mapPartitions(my_partition_func)

rdd_map_partition.collect()

[19, 19, 36, 98]

In [30]:
# union

rdd2 = spark.sparkContext.parallelize(random.sample(range(1,20), 10),2)

rdd2.collect()


[14, 8, 17, 16, 13, 3, 10, 15, 5, 12]

In [31]:
rdd1.union(rdd2).collect()

[15, 4, 14, 5, 25, 11, 35, 7, 19, 37, 14, 8, 17, 16, 13, 3, 10, 15, 5, 12]

In [32]:
rdd_union = rdd1.union(rdd2)

rdd_union.glom().collect()

[[15, 4],
 [14, 5],
 [25, 11],
 [35, 7, 19, 37],
 [14, 8, 17, 16, 13],
 [3, 10, 15, 5, 12]]

In [33]:
rdd1.union(rdd2).getNumPartitions()

6

In [34]:
rdd1.union(rdd2).glom().collect()

[[15, 4],
 [14, 5],
 [25, 11],
 [35, 7, 19, 37],
 [14, 8, 17, 16, 13],
 [3, 10, 15, 5, 12]]

In [35]:
rdd_inter = rdd1.intersection(rdd2)

rdd_inter.collect()

[14, 15, 5]

In [36]:
rdd_inter.glom().collect()

[[], [], [14], [15], [], [5]]

In [37]:
rdd_inter.coalesce(1).glom().collect()

[[14, 15, 5]]

In [38]:
# takeSample withreplacement, num, [seed]

rdd1.takeSample(False, 5)

[11, 4, 5, 37, 14]

In [39]:
# takeOrdered

rdd1.takeOrdered(5)

[4, 5, 7, 11, 14]

In [40]:
rdd1.takeOrdered(5, key=lambda x: -x)

[37, 35, 25, 19, 15]

In [41]:
rdd1.reduce(lambda x, y: x+y)

172

In [42]:
rdd_rbk = rdd_union.map(lambda x: (x, x*x)).reduceByKey(lambda x,y: x+y)

In [43]:
rdd_rbk.toDF(schema=["key","value"]).show(vertical=False,truncate=False)

+---+-----+
|key|value|
+---+-----+
|12 |144  |
|25 |625  |
|7  |49   |
|19 |361  |
|37 |1369 |
|13 |169  |
|14 |392  |
|8  |64   |
|15 |450  |
|3  |9    |
|4  |16   |
|16 |256  |
|10 |100  |
|5  |50   |
|11 |121  |
|35 |1225 |
|17 |289  |
+---+-----+



In [44]:
# sortByKey()

rdd_rbk.sortByKey().collect()

[(3, 9),
 (4, 16),
 (5, 50),
 (7, 49),
 (8, 64),
 (10, 100),
 (11, 121),
 (12, 144),
 (13, 169),
 (14, 392),
 (15, 450),
 (16, 256),
 (17, 289),
 (19, 361),
 (25, 625),
 (35, 1225),
 (37, 1369)]

In [45]:
rdd_rbk.countByKey()

defaultdict(int,
            {12: 1,
             25: 1,
             7: 1,
             19: 1,
             37: 1,
             13: 1,
             14: 1,
             8: 1,
             15: 1,
             3: 1,
             4: 1,
             16: 1,
             10: 1,
             5: 1,
             11: 1,
             35: 1,
             17: 1})

In [46]:
rdd_gbk = rdd_rbk.groupByKey().collect()

In [47]:
for item in rdd_gbk:

  print(item[0], list(item[1]))

12 [144]
25 [625]
7 [49]
19 [361]
37 [1369]
13 [169]
14 [392]
8 [64]
15 [450]
3 [9]
4 [16]
16 [256]
10 [100]
5 [50]
11 [121]
35 [1225]
17 [289]


In [48]:
rdd_gby_1 = rdd_rbk.groupByKey()

In [49]:
import os

# os.environ["PYTHONHASHSEED"] = "false"

In [50]:


rdd_rbk.sortByKey().lookup(10)

[100]

In [51]:
rdd_rbk.cache()

PythonRDD[94] at RDD at PythonRDD.scala:53

In [52]:
  rdd_rbk.is_cached

True

In [53]:
rdd_rbk.unpersist()

PythonRDD[94] at RDD at PythonRDD.scala:53

In [54]:
# DF

spark

In [55]:
#df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/kaggle/input/fifa-18-demo-player-dataset/CompleteDataset.csv")

In [56]:
#df.printSchema()

In [57]:
#df.show(truncate=False)

In [58]:
#df.count()

In [59]:
#df.rdd.glom().collect()

In [60]:
#df.rdd.getNumPartitions()

In [61]:
#df2 = df.repartition(4)

In [62]:
#df2.rdd.getNumPartitions()

In [63]:
#df2.rdd.glom().collect()

In [64]:
#df.withColumnRenamed("_c0", "id").show()

In [65]:
#df2 = df.withColumnRenamed("_c0", "id")


In [66]:
#df2.printSchema()

In [67]:
#df2.rdd.getNumPartitions()

In [68]:
#df2 = df2.repartition(6)

In [69]:
#df2.rdd.getNumPartitions()

In [70]:
#df3 = df2.repartition(3, "Nationality")

In [71]:
#df3.rdd.glom().collect()

In [72]:
#df3.rdd.getNumPartitions()

In [73]:
#df3.show()

In [74]:
#df3.na.fill({"CAM": 10, "CB": 1}).show()

In [75]:
#df3.filter(df3["Nationality"] == "Germany").show()

In [76]:
#df3.groupBy("Nationality").count().sort("Nationality").show(50)

In [77]:
#df3.groupBy("Nationality", "Age").count().sort("Nationality","Age").show(50)

In [78]:
from pyspark.sql import *

In [79]:
# visualize with pandas

#df3_res = df3.where(df3["Overall"]>70).filter(df3["Nationality"] == "Albania").groupBy("Nationality", "Age").count().sort("Nationality","Age").toPandas()

#df3_res.plot(kind="bar", x="Nationality", y="count")


In [80]:
#df3_res_2 = df3.where(df3["Overall"]>70).groupBy("Age").count().sort("Age").toPandas()

#df3_res_2.plot(kind="bar", x="Age", y="count")


In [81]:
# UDF

def uppercase_lowercase(str):

  if str:  
      if len(str) > 10:
    
        return str.upper()
    
      else:
    
        return str.lower()
  else:
      return ""



spark.udf.register("uppercase_lowercase", uppercase_lowercase, StringType())

<function __main__.uppercase_lowercase(str)>

In [82]:
#df = spark.read.format("json").load('/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json')

In [83]:
# register df

#df.createOrReplaceTempView("fifa_table")

In [84]:
# df2.select("Name", uppercase_lowercase("Name")).show()

#spark.sql("select comments, uppercase_lowercase(comments) as upper_comments from fifa_table").show()

In [85]:
#broadcast(df)

In [86]:
#df.select("abstract", uppercase_lowercase("abstract")).show()

In [87]:
#df.printSchema()

In [88]:
#df.count()

In [89]:
#df.rdd.getNumPartitions()

In [90]:
#df.rdd.glom().collect()

In [91]:
#df.show()

In [92]:
#df.cache()

In [93]:
spark.sparkContext.defaultParallelism

4

In [94]:
#df.count()

In [95]:
import json

In [96]:
rdd_json = spark.sparkContext.textFile('/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json')

In [97]:
rdd = rdd_json.map(lambda x: json.loads(x))

In [98]:
rdd

PythonRDD[97] at RDD at PythonRDD.scala:53

In [99]:
spark.sparkContext.defaultParallelism

4

In [100]:
# rdd.persist()

In [101]:
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

In [102]:
rdd.count()

2011231

In [103]:
rdd.flatMap(lambda x: x.keys()).distinct().collect()

['comments',
 'versions',
 'abstract',
 'report-no',
 'license',
 'update_date',
 'authors',
 'journal-ref',
 'doi',
 'categories',
 'submitter',
 'authors_parsed',
 'id',
 'title']

In [104]:
rdd.map(lambda x: x['license']).distinct().collect()

[None,
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/licenses/by/3.0/']

In [105]:
# min title

min_title = rdd.map(lambda x: x['title']).distinct().reduce(lambda x,y: x if (len(x)<len(y)) else y)

print(min_title)

Q


In [106]:
# max title

max_title = rdd.map(lambda x: x['title']).distinct().reduce(lambda x,y: x if (len(x)>len(y)) else y)

print(max_title)

Investigation of the 2-body system with a rotating central body (e. g.
  earth-moon system) within the Projective Unified Field theory: the transfer
  of rotational angular momentum and energy from the central body to the
  orbital 2-body system, the tidal and the non-tidal influences (mechanical,
  general-relativistic Lense-Thirring effect and cosmological
  PUFT-contributions)


In [107]:
import re

In [108]:
# find abbreeviations with 5 or more letters in the "abstract" column

# exept : [^_! /\\:;&#,~<>]

regx = r"\((\w[^_! /\\:;&#,~?${}<>]{5,})\)"

In [109]:
print(re.search(regx, "of the Large Hadron\nCollider (LHCOPS)"))

<re.Match object; span=(29, 37), match='(LHCOPS)'>


In [110]:


rdd.filter(lambda x: re.search(regx, x['abstract'])).count()

188806

In [111]:
# rdd.filter(lambda x: re.search(regx, x['abstract'])).collect()

In [112]:
# Number of archive per month

rdd.map(lambda x: (x['update_date'][5:7], 1)).reduceByKey(lambda x,y: x+y).collect()

[('01', 134247),
 ('06', 191746),
 ('05', 296587),
 ('10', 197755),
 ('11', 297963),
 ('02', 116948),
 ('04', 117126),
 ('03', 126458),
 ('07', 122649),
 ('09', 138978),
 ('08', 138469),
 ('12', 132305)]

In [113]:
from datetime import datetime



def extract_date(line):

  d = datetime.strptime(line, '%Y-%m-%d')

  return d.month



extract_date("2008-12-20")

12

In [114]:
# Number of archive per month

rdd.map(lambda x: (extract_date(x['update_date']), 1)).reduceByKey(lambda x,y: x+y).collect()

[(1, 134247),
 (2, 116948),
 (3, 126458),
 (4, 117126),
 (5, 296587),
 (6, 191746),
 (7, 122649),
 (8, 138469),
 (9, 138978),
 (10, 197755),
 (11, 297963),
 (12, 132305)]

In [115]:
# the average number of pages

# in comments column

# extract only the number be carefull with "None" value

pages = "pages"



def extract_num_pages(comments):

  regex = r"(\d+ pages)"

  r = re.findall(regex, str(comments))

  if r:

   return int(r[0].split(' ')[0])

  return 0



extract_num_pages("Extensively revised version. In the revised version, we have\n  discussed the effect of viscosity on (quark) transverse momentum distribution\n  and on elliptic flow. It was shown that elliptic flow saturates due to\n  non-equilirium correction to the equilibrium distribution function and can\n  not be mimicked by ideal dynamics. 15 pages, 16 figures")

15

In [116]:
rdd_com = rdd.map(lambda x: (x["comments"],extract_num_pages(x["comments"]))).collect()


In [117]:
rdd.map(lambda x: extract_num_pages(x['comments'])).mean()

10.51073496778838

In [118]:
rdd_page_row = rdd.map(lambda x: (x["comments"],extract_num_pages(x["comments"]))).count()

In [119]:
rdd_page_row

2011231

In [120]:
rdd_nn_pages = rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).count()

In [121]:
rdd_nn_pages

2011231

In [122]:
rdd_filter_zero = rdd.filter(lambda x: x != 0).count()

In [123]:
rdd_filter_zero

2011231

In [124]:
# average with mean function
rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).mean()

10.51073496778838

In [125]:
# average with reduce
rdd_reduce = rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).reduce(lambda x,y: x+y)

In [126]:
print(rdd_reduce/rdd_filter_zero)

10.510734967788384


In [127]:
# using DF
df = spark.read.json("/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json")

In [128]:
df

DataFrame[abstract: string, authors: string, authors_parsed: array<array<string>>, categories: string, comments: string, doi: string, id: string, journal-ref: string, license: string, report-no: string, submitter: string, title: string, update_date: string, versions: array<struct<created:string,version:string>>]

In [129]:
df.rdd.getNumPartitions()

25

In [130]:
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [131]:
df.count()

2011231

In [132]:
from pyspark.sql.types import *

schema = StructType([
    StructField("authors",StringType(), True),
    StructField("categories",StringType(), True),
    StructField("license",StringType(), True),
    StructField("comments",StringType(), True),
    StructField("abstract",StringType(), True),
    StructField("versions", ArrayType(StringType()), True)
])
schema

StructType([StructField('authors', StringType(), True), StructField('categories', StringType(), True), StructField('license', StringType(), True), StructField('comments', StringType(), True), StructField('abstract', StringType(), True), StructField('versions', ArrayType(StringType(), True), True)])

In [133]:
# Binding data to a schama
df = spark.read.json("/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json", schema=schema)

In [134]:
df.show(vertical=True)

-RECORD 0--------------------------
 authors    | C. Bal\'azs, E. L... 
 categories | hep-ph               
 license    | NULL                 
 comments   | 37 pages, 15 figu... 
 abstract   |   A fully differe... 
 versions   | [{"version":"v1",... 
-RECORD 1--------------------------
 authors    | Ileana Streinu an... 
 categories | math.CO cs.CG        
 license    | http://arxiv.org/... 
 comments   | To appear in Grap... 
 abstract   |   We describe a n... 
 versions   | [{"version":"v1",... 
-RECORD 2--------------------------
 authors    | Hongjun Pan          
 categories | physics.gen-ph       
 license    | NULL                 
 comments   | 23 pages, 3 figures  
 abstract   |   The evolution o... 
 versions   | [{"version":"v1",... 
-RECORD 3--------------------------
 authors    | David Callan         
 categories | math.CO              
 license    | NULL                 
 comments   | 11 pages             
 abstract   |   We show that a ... 
 versions   | [{"version":"v

In [135]:
# df count
df.count()

2011231

In [136]:
# Missing values for "comments" and "licence" attributes
# drop
df = df.dropna(subset = ["comments"])

df.count()

1567634

In [137]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|          categories|             license|            comments|            abstract|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|              hep-ph|                NULL|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|      physics.gen-ph|                NULL| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|             math.CO|                NULL|            11 pages|  We show that a ...|[{"version":"v1",...|
|Y. H. Pong and C....|   cond-mat.mes-hall|                NULL|6 pages, 4 figure...|  We study the tw..

In [138]:
# Missing values for "comments" and "licence" attributes

# replace
df = df.fillna(value = "Unknown", subset = ["license"])
df.count()

1567634

In [139]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|          categories|             license|            comments|            abstract|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|              hep-ph|             Unknown|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|      physics.gen-ph|             Unknown| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|             math.CO|             Unknown|            11 pages|  We show that a ...|[{"version":"v1",...|
|Y. H. Pong and C....|   cond-mat.mes-hall|             Unknown|6 pages, 4 figure...|  We study the tw..

In [140]:
# renamed df to reuse in sql query
df.createOrReplaceTempView("Archive")

query = """select authors, categories from Archive
            where categories like 'math%'
"""
spark.sql(query).show()
print(spark.sql(query).count())

+--------------------+--------------------+
|             authors|          categories|
+--------------------+--------------------+
|Ileana Streinu an...|       math.CO cs.CG|
|        David Callan|             math.CO|
|  Sergei Ovchinnikov|             math.CO|
|Clifton Cunningha...|     math.NT math.AG|
|        Koichi Fujii|     math.CA math.AT|
|         Norio Konno|     math.PR math.AG|
|Simon J.A. Malham...|             math.NA|
|Robert P. C. de M...|             math.RA|
|  P\'eter E. Frenkel|     math.CA math.PR|
|          Mihai Popa|     math.OA math.FA|
|   Debashish Goswami|math.QA math-ph m...|
|      Mikkel {\O}bro|             math.CO|
|Nabil L. Youssef,...|       math.DG gr-qc|
|         Boris Rubin|             math.FA|
|         A. I. Molev|     math.AG math.CO|
| Branko J. Malesevic|     math.NT math.CV|
|   John W. Robertson|             math.DS|
|     Yu.N. Kosovtsov|     math-ph math.MP|
|        Osamu Fujino|             math.AG|
|Stephen C. Power ...|          

304590


In [141]:
# Get licences with 5 or more letters in the "abstract"
regexp1 = r"([a-z A-Z]{5,})"

def fiveOrMoreLetters(x):
    if str(x["abstract"]):
        r = re.search(regexp1,x["abstract"])
        if r:
            return x["license"]
        else:
            return None
    else:
        return None

in_js = { 'license': 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
          'abstract': '  Given an abelian algebraic group $A$ over a global field $F$, $\\alpha \\in\nA(F)$.\n'}
fiveOrMoreLetters(in_js)

'http://arxiv.org/licenses/nonexclusive-distrib/1.0/'

In [142]:
df_license = df.withColumn("regxp", f.lit(regexp1)).select(df.license).where(f.regexp("abstract","regxp"))

In [143]:
df_license.show()

+--------------------+
|             license|
+--------------------+
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
+--------------------+
only showing top 20 rows



In [144]:
# 2nd method
query = """select license from Archive
            where abstract REGEXP "([a-z A-Z]{5,})"
"""
spark.sql(query).show()

+--------------------+
|             license|
+--------------------+
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
+--------------------+
only showing top 20 rows



In [145]:
# extract the statistics of numbers of pages
spark.udf.register("extract_num_pages", extract_num_pages, IntegerType())
query = """select extract_num_pages(comments) as num_pages from Archive
            where abstract is not null
"""
df_num_pages = spark.sql(query)
df_num_pages.show()
df_num_pages.createOrReplaceTempView("NumPages")
query1 = """select sum(num_pages)/count(*) as avg from NumPages
            where num_pages != 0
"""
spark.sql(query1).show()
query2 = """select AVG(extract_num_pages(comments)) as avg, SUM(extract_num_pages(comments)) as sum,
            STD(extract_num_pages(comments)) as std
            from Archive
            where license = 'Unknown'
"""
spark.sql(query2).show()

+---------+
|num_pages|
+---------+
|       37|
|        0|
|       23|
|       11|
|        6|
|       16|
|        0|
|       36|
|       14|
|       18|
|       22|
|       17|
|       10|
|       20|
|        6|
|       21|
|        5|
|       20|
|        4|
|        0|
+---------+
only showing top 20 rows



+-----------------+
|              avg|
+-----------------+
|17.85319004286046|
+-----------------+



+------------------+-------+------------------+
|               avg|    sum|               std|
+------------------+-------+------------------+
|13.368011068572079|5642584|16.777518213632323|
+------------------+-------+------------------+



In [146]:
#  'versions': [{'version': 'v1', 'created': 'Sat, 31 Mar 2007 02:26:18 GMT'},
   # {'version': 'v2', 'created': 'Sat, 13 Dec 2008 17:26:00 GMT'}],

# Question 9: Get the average number of pages per day for version 1

# df.select(extract_num_pages(col("comments")) ).where(get_json_object(col("versions")[0], "$.version") == "v1").show()


In [147]:
# extract day from created field
def extract_day(created_datetime):
    if created_datetime:
        return created_datetime.split(',')[0]
    else:
        return None

extract_day('Thu, 5 Apr 2007 04:48:39 GMT')

'Thu'

In [148]:
spark.udf.register("extract_day", extract_day, StringType())

<function __main__.extract_day(created_datetime)>

In [149]:

query_avg_pages_per_day = """select AVG(extract_num_pages(comments)) as avg_pages_per_day_v1 , extract_day(get_json_object(versions[0], "$.created")) as day
                             from Archive
                             where get_json_object(versions[0], "$.version") == "v1"
                             group by day
"""
spark.sql(query_avg_pages_per_day).show(truncate=False)

+--------------------+---+
|avg_pages_per_day_v1|day|
+--------------------+---+
|13.438900236335833  |Sun|
|13.647621634042908  |Mon|
|13.554587235518557  |Thu|
|13.063888425848752  |Sat|
|13.473081449845305  |Wed|
|13.470779008622024  |Tue|
|13.418742136828348  |Fri|
+--------------------+---+



In [150]:
# Read and load Data to spark
# data source http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

rdd = spark.sparkContext.textFile("/kaggle/input/kddcup/kddcup.data")

In [151]:
# cache repartitions
rdd.getNumPartitions()

23

In [152]:
spark.sparkContext.defaultParallelism

4

In [153]:
# Q1: get ten records randomly
rdd.takeSample(False, 10, 1234)

['0,tcp,smtp,SF,2224,326,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,193,0.97,0.05,0.03,0.02,0.00,0.01,0.00,0.00,normal.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,5

In [154]:
rdd.take(2)

['0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal.']

In [155]:
# Q2: rdd count elements
rdd.count()

4898431

In [156]:
# Q3: Calculate the ratio of "normal" connections
normal_rdd = rdd.filter(lambda x: "normal." in x)

In [157]:
normal_rdd.count()

972781

In [158]:
ratio = normal_rdd.count()/rdd.count()

In [159]:
print(f"Ratio of normal connection: {format(round(ratio, 4) * 100)} %")

Ratio of normal connection: 19.86 %


In [160]:
# Q4: Get the list of labels
list_of_labels = rdd.map(lambda x: x.split(',')[-1]).distinct().collect()
list_of_labels

['spy.',
 'land.',
 'warezclient.',
 'phf.',
 'back.',
 'smurf.',
 'teardrop.',
 'rootkit.',
 'portsweep.',
 'imap.',
 'perl.',
 'pod.',
 'nmap.',
 'guess_passwd.',
 'ipsweep.',
 'ftp_write.',
 'buffer_overflow.',
 'loadmodule.',
 'multihop.',
 'normal.',
 'warezmaster.',
 'neptune.',
 'satan.']

In [161]:
# Q5: Count the number of connections for each label
%time
conn_num_labels = rdd.map(lambda x: (x.split(',')[-1],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).collect()
conn_num_labels

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs


[('smurf.', 2807886),
 ('neptune.', 1072017),
 ('normal.', 972781),
 ('satan.', 15892),
 ('ipsweep.', 12481),
 ('portsweep.', 10413),
 ('nmap.', 2316),
 ('back.', 2203),
 ('warezclient.', 1020),
 ('teardrop.', 979),
 ('pod.', 264),
 ('guess_passwd.', 53),
 ('buffer_overflow.', 30),
 ('land.', 21),
 ('warezmaster.', 20),
 ('imap.', 12),
 ('rootkit.', 10),
 ('loadmodule.', 9),
 ('ftp_write.', 8),
 ('multihop.', 7),
 ('phf.', 4),
 ('perl.', 3),
 ('spy.', 2)]

In [162]:
%time
rdd.map(lambda x: (x.split(',')[-1],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).toDF(schema=["Attack_Name", "Rate"]).show(50,truncate=False)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


+----------------+-------+
|Attack_Name     |Rate   |
+----------------+-------+
|smurf.          |2807886|
|neptune.        |1072017|
|normal.         |972781 |
|satan.          |15892  |
|ipsweep.        |12481  |
|portsweep.      |10413  |
|nmap.           |2316   |
|back.           |2203   |
|warezclient.    |1020   |
|teardrop.       |979    |
|pod.            |264    |
|guess_passwd.   |53     |
|buffer_overflow.|30     |
|land.           |21     |
|warezmaster.    |20     |
|imap.           |12     |
|rootkit.        |10     |
|loadmodule.     |9      |
|ftp_write.      |8      |
|multihop.       |7      |
|phf.            |4      |
|perl.           |3      |
|spy.            |2      |
+----------------+-------+



In [186]:
# Q6: get the connection type with successfull 'root_shell = 0' connections to servers, 
# where the number of data bytes from source [src_bytes = 500] times more than from server ()

def data_500_times_more(x,y) -> bool:
    src_bytes = int(x)
    srv_bytes = int(y)
    if srv_bytes != 0:
        if ((srv_bytes/src_bytes) > 500):
            return True
        else:
            return False
    else:
        return (src_bytes >= 500)

   

data_500_times_more('296','507534')

True

In [164]:
def is_success_conn(x) -> bool:
    if x:
        s_conn = int(x.split(',')[13])
        if s_conn == 1:
            return True
        else:
            False
    else:
        False

is_success_conn('0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.')

True

In [165]:
import gc
gc.collect()

528

In [185]:
%time
rdd_conn_type = rdd.filter(lambda x: x.split(",")[13] == '1').map(lambda x: (x.split(",")[1], x.split(",")[4], x.split(",")[5])).filter(lambda x: int(x[2]) >  int(x[1]) * 500).collect()
rdd_conn_type

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


[('tcp', '296', '507534'),
 ('tcp', '296', '507534'),
 ('tcp', '266', '507534'),
 ('tcp', '296', '507534'),
 ('tcp', '353', '759161'),
 ('tcp', '351', '759161'),
 ('tcp', '246', '866032'),
 ('tcp', '317', '394616'),
 ('tcp', '262', '744605'),
 ('tcp', '173', '744605'),
 ('tcp', '262', '744605'),
 ('tcp', '255', '574784'),
 ('tcp', '433', '1524348'),
 ('tcp', '1794', '3851730'),
 ('tcp', '0', '2072'),
 ('tcp', '224', '2776333'),
 ('tcp', '0', '2072'),
 ('tcp', '465', '320362'),
 ('tcp', '0', '2072'),
 ('tcp', '0', '2072')]

In [214]:
# Q7: Get the list of Protocols that are normal and vulnerable to attacks, where there is NOT guest_login to destination addresses
rdd_protos = rdd.filter(lambda x: 'normal.' in x.split(",")[-1] and int(x.split(',')[21]) == 0).map(lambda x: (x.split(',')[1],1)).reduceByKey(lambda x,y: x+y).collect()
rdd_protos

[('tcp', 764894), ('udp', 191348), ('icmp', 12763)]

In [217]:
# Attacks by protocol and label
%time
rdd_proto_label = rdd.filter(lambda x: 'normal.' not in x.split(",")[-1] and int(x.split(',')[21]) == 0).map(lambda x: ((x.split(',')[1],x.split(",")[-1]),1)).reduceByKey(lambda x,y: x+y).sortByKey().collect()
rdd_proto_label


CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.68 µs


[(('icmp', 'ipsweep.'), 11557),
 (('icmp', 'nmap.'), 1032),
 (('icmp', 'pod.'), 264),
 (('icmp', 'portsweep.'), 6),
 (('icmp', 'satan.'), 37),
 (('icmp', 'smurf.'), 2807886),
 (('tcp', 'back.'), 2203),
 (('tcp', 'buffer_overflow.'), 30),
 (('tcp', 'ftp_write.'), 6),
 (('tcp', 'guess_passwd.'), 52),
 (('tcp', 'imap.'), 12),
 (('tcp', 'ipsweep.'), 924),
 (('tcp', 'land.'), 21),
 (('tcp', 'loadmodule.'), 9),
 (('tcp', 'multihop.'), 5),
 (('tcp', 'neptune.'), 1072017),
 (('tcp', 'nmap.'), 1034),
 (('tcp', 'perl.'), 3),
 (('tcp', 'phf.'), 4),
 (('tcp', 'portsweep.'), 10407),
 (('tcp', 'rootkit.'), 7),
 (('tcp', 'satan.'), 14146),
 (('tcp', 'spy.'), 2),
 (('tcp', 'warezclient.'), 713),
 (('tcp', 'warezmaster.'), 18),
 (('udp', 'nmap.'), 250),
 (('udp', 'rootkit.'), 3),
 (('udp', 'satan.'), 1708),
 (('udp', 'teardrop.'), 979)]

In [223]:
# Q8: Get summary statistics for the sum of connections to the same destination IP address
%time
from pyspark.mllib.stat import Statistics
from math import sqrt

rdd_summary = rdd.filter(lambda x: 'tcp' in x.split(",")[1]).map(lambda x: (x.split(',')[1],int(x.split(",")[-10]))).reduceByKey(lambda x,y: x+y).sortByKey().collect()
print(rdd_summary)



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


[('tcp', 179509939)]


In [227]:
summary = Statistics.colStats(rdd.filter(lambda x: 'tcp' in x.split(",")[1]).map(lambda x: [int(x.split(",")[-11])]))

print(f"mean : {round(float(summary.mean()),3)}")
print(f"variance: {round(float(summary.variance()),3)}")
print(f"ecart-type: {round(float(sqrt(summary.variance())),3)}")
print(f"min: {round(float(summary.min()),3)}")
print(f"max: {round(float(summary.max()),3)}")
summary.

mean : 201.752
variance: 8231.163
ecart-type: 90.726
min: 0.0
max: 255.0


/tmp/ipykernel_30/4101549886.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"mean : {round(float(summary.mean()),3)}")
/tmp/ipykernel_30/4101549886.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"variance: {round(float(summary.variance()),3)}")
/tmp/ipykernel_30/4101549886.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"ecart-type: {round(float(sqrt(summary.variance())),3)}")
/tmp/ipykernel_30/4101549886.py:6: DeprecationWarning:

In [228]:
# Q9: Challenge : Filter number of icmp-based attacks for each service
%time
rdd_icmp_attack_by_service = rdd.filter(lambda x: 'normal.' not in x.split(",")[-1] and "icmp" in x.split(',')[1]).map(lambda x: ((x.split(',')[1],x.split(",")[2]),1)).reduceByKey(lambda x,y: x+y).sortByKey().collect()
rdd_icmp_attack_by_service


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


[(('icmp', 'eco_i'), 12570),
 (('icmp', 'ecr_i'), 2808204),
 (('icmp', 'tim_i'), 5),
 (('icmp', 'urp_i'), 3)]

In [229]:
df = spark.read.text("/kaggle/input/kddcup/kddcup.data")

In [243]:
df.show(2,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                             |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
|0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.|
|0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal. |
+--------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [244]:
import pyspark.sql.functions as f

split_col = f.split(df['value'], ',')
cols = ["duration",
"protocol_type",
"service",
"flag",
"src_bytes",
"dst_bytes",
"land",
"wrong_fragment",
"urgent",
"hot",
"num_failed_logins",
"logged_in",
"num_compromised",
"root_shell",
"su_attempted",
"num_root",
"num_file_creations",
"num_shells",
"num_access_files",
"num_outbound_cmds",
"is_host_login",
"is_guest_login",
"count",
"srv_count",
"serror_rate",
"srv_serror_rate",
"rerror_rate",
"srv_rerror_rate",
"same_srv_rate",
"diff_srv_rate",
"srv_diff_host_rate",
"dst_host_count",
"dst_host_srv_count",
"dst_host_same_srv_rate",
"dst_host_diff_srv_rate",
"dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate",
"dst_host_serror_rate",
"dst_host_srv_serror_rate",
"dst_host_rerror_rate",
"dst_host_srv_rerror_rate",
"label"]

df_f = (df.withColumn(cols[0], split_col[0]).withColumn(cols[1], split_col[1]).withColumn(cols[2], split_col[2]).withColumn(cols[3], split_col[3])
        .withColumn(cols[4], split_col[4]).withColumn(cols[5], split_col[5]).withColumn(cols[6], split_col[6]).withColumn(cols[7], split_col[7])
        .withColumn(cols[8], split_col[8]).withColumn(cols[9], split_col[9]).withColumn(cols[10], split_col[10]).withColumn(cols[11], split_col[11])
        .withColumn(cols[12], split_col[12])
        .withColumn(cols[13], split_col[13]).withColumn(cols[14], split_col[14]).withColumn(cols[15], split_col[15]).withColumn(cols[16], split_col[16])
        .withColumn(cols[17], split_col[17]).withColumn(cols[18], split_col[18]).withColumn(cols[19], split_col[19]).withColumn(cols[20], split_col[20])
        .withColumn(cols[21], split_col[21]).withColumn(cols[22], split_col[22]).withColumn(cols[23], split_col[23]).withColumn(cols[24], split_col[24])
        .withColumn(cols[26], split_col[26]).withColumn(cols[27], split_col[27]).withColumn(cols[28], split_col[28]).withColumn(cols[29], split_col[29])
        .withColumn(cols[30], split_col[30]).withColumn(cols[31], split_col[31]).withColumn(cols[32], split_col[32]).withColumn(cols[33], split_col[33])
        .withColumn(cols[34], split_col[34]).withColumn(cols[35], split_col[35]).withColumn(cols[36], split_col[36]).withColumn(cols[37], split_col[37])
        .withColumn(cols[38] , split_col[38]).withColumn(cols[39], split_col[39]).withColumn(cols[40], split_col[40]).withColumn(cols[41], split_col[41])
        .drop("value")
       )

In [245]:
df_f.show(1,vertical=True,truncate=False)

-RECORD 0------------------------------
 duration                    | 0       
 protocol_type               | tcp     
 service                     | http    
 flag                        | SF      
 src_bytes                   | 215     
 dst_bytes                   | 45076   
 land                        | 0       
 wrong_fragment              | 0       
 urgent                      | 0       
 hot                         | 0       
 num_failed_logins           | 0       
 logged_in                   | 1       
 num_compromised             | 0       
 root_shell                  | 0       
 su_attempted                | 0       
 num_root                    | 0       
 num_file_creations          | 0       
 num_shells                  | 0       
 num_access_files            | 0       
 num_outbound_cmds           | 0       
 is_host_login               | 0       
 is_guest_login              | 0       
 count                       | 1       
 srv_count                   | 1       
